### Metaprogramming - Application 1

Are you tired of writing boiler-plate code like this:

In [1]:
class Point2D:
    __slots__ = ('_x', '_y')
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    def __eq__(self, other):
        return isinstance(other, Point) and (self.x, self.y) == (other.x, other.y)
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'
    
    def __str__(self):
        return f'({self.x}, {self.y})'
        
class Point3D:
    __slots__ = ('_x', '_y', '_z')
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z
    
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    @property
    def z(self):
        return self._z
    
    def __eq__(self, other):
        return isinstance(other, Point) and (self.x, self.y, self.z) == (other.x, other.y, other.z)
    
    def __hash__(self):
        return hash((self.x, self.y, self.z))

    def __repr__(self):
        return f'Point2D({self.x}, {self.y}, {self.z})'
    
    def __str__(self):
        return f'({self.x}, {self.y}, {self.z})'


**Solution:**

In [12]:
# metaclass
class SlottedStruct(type):
    def __init__(cls, name, bases, dct):
        super().__init__(name, bases, dct)

    def __new__(cls, name, bases, dct):
        dimensions = int(name[-2]) if name[-2].isdigit() else 2

        dct['__slots__'] = tuple(f'_coord{i}' for i in range(dimensions))
        
        return super().__new__(cls, name, bases, dct)

    def __repr__(cls):
        dimensions = int(cls.__name__[-2]) if cls.__name__[-2].isdigit() else 2
        
        return f"{cls.__name__}({', '.join(str(getattr(cls, f'_coord{i}')) for i in range(dimensions))})"

    def __str__(cls):
        dimensions = int(cls.__name__[-2]) if cls.__name__[-2].isdigit() else 2
        
        return f"({', '.join(str(getattr(cls, f'_coord{i}')) for i in range(dimensions))})"

    def __eq__(cls, other):
        dimensions = int(cls.__name__[-2]) if cls.__name__[-2].isdigit() else 2
        
        if not isinstance(other, cls):
            return False
        
        return all(getattr(cls, f'_coord{i}') == getattr(other, f'_coord{i}') for i in range(dimensions))

    def __hash__(cls):
        dimensions = int(cls.__name__[-2]) if cls.__name__[-2].isdigit() else 2
        
        return hash(tuple(getattr(cls, f'_coord{i}') for i in range(dimensions)))


class Point2D(metaclass=SlottedStruct):
    def __init__(self, x, y):
        self._coord0 = x
        self._coord1 = y

    @property
    def x(self):
        return self._coord0

    @property
    def y(self):
        return self._coord1


class Point3D(metaclass=SlottedStruct):
    def __init__(self, x, y, z):
        self._coord0 = x
        self._coord1 = y
        self._coord2 = z

    @property
    def x(self):
        return self._coord0

    @property
    def y(self):
        return self._coord1

    @property
    def z(self):
        return self._coord2


class Point4D(metaclass=SlottedStruct):
    def __init__(self, x, y, z, w):
        self._coord0 = x
        self._coord1 = y
        self._coord2 = z
        self._coord3 = w

    @property
    def x(self):
        return self._coord0

    @property
    def y(self):
        return self._coord1

    @property
    def z(self):
        return self._coord2

    @property
    def w(self):
        return self._coord3


class Point5D(metaclass=SlottedStruct):
    def __init__(self, x, y, z, w, v):
        self._coord0 = x
        self._coord1 = y
        self._coord2 = z
        self._coord3 = w
        self._coord4 = v

    @property
    def x(self):
        return self._coord0

    @property
    def y(self):
        return self._coord1

    @property
    def z(self):
        return self._coord2

    @property
    def w(self):
        return self._coord3

    @property
    def v(self):
        return self._coord4


point2d = Point2D(1, 2)
pint2d_ = Point2D(2, 3)
point3d = Point3D(1, 2, 3)
point4d = Point4D(1, 2, 3, 4)
point5d = Point5D(1, 2, 3, 4, 5)

print(point2d) 
print(point3d)  
print(point4d)  
print(point5d, "\n")  


print(hash(point2d))  
print(hash(point3d))  
print(hash(point4d))  
print(hash(point5d))  


288924585
290297696
290293456
288555752


*Brief explanation*: 

As we can see above the new created metaclass helps us to overcome the problem of redundancy as all the necessary functions are kept inside it, so there is no need to implement the same functions for multiple times for new points, we just need to inherit the methods from the general metaclass.